In [2]:
# import neccesary libraries

import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

In [3]:
# Set the height and width of the camera frame

height = 480
width = 1280

In [4]:
# Video capture object read from the camera

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)

True

In [5]:
# Create a separate window with the desired dimensions
output_height = 480
output_width = 640
cv2.namedWindow("Output Screen", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Output Screen", output_width, output_height)

In [6]:
# object of HandDetector

detector = HandDetector(detectionCon=0.8, maxHands=1)

In [7]:
# anootation variable

annotation = [[]]
annotationStart = False
annotationNumber = -1

In [8]:
shapeName = ""

In [9]:
def shapeClassification(contour):
    # Approximate the contour to determine the shape

    perimeter = cv2.arcLength(contour, True)

    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

    # Classify the shape based on the number of sides

    if len(approx) == 3:
        return "Triangle"

    elif len(approx) == 4:
        x, y, w, h = cv2.boundingRect(approx)
        aspectRatio = float(w) / h

        if 0.95 <= aspectRatio <= 1.05:
            return "Square"
        else:
            return "Rectangle"

    elif len(approx) == 5:
        return "Pentagon"

    else:
        return "Circle"

In [10]:
while True:

    success, img = cap.read()

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img)

    # Finger detection
    
    if hands:
        hand = hands[0]
        fingers = detector.fingersUp(hand)
        lmList = hand['lmList']

        # Value for easier drawing
        yVal = int(np.interp(lmList[8][1], [100, height - 100], [0, output_height]))
        indexFinger = lmList[8][0], yVal

        # Pointer show
        if fingers == [0, 1, 1, 0, 0]:
            cv2.circle(img, indexFinger, 12, (0, 0, 255), cv2.FILLED)

        # Draw pointer
        if fingers == [0, 1, 0, 0, 0]:
            if annotationStart is False:
                annotationStart = True
                annotationNumber += 1
                annotation.append([])
            cv2.circle(img, indexFinger, 12, (0, 0, 255), cv2.FILLED)
            annotation[annotationNumber].append(indexFinger)
        else:
            if annotationStart:
                annotationStart = False
                if len(annotation[annotationNumber]) > 2:
                    # Convert the annotation to a NumPy array and find the contour
                    contour = np.array(annotation[annotationNumber], dtype=np.int32)
                    cv2.drawContours(img, [contour], 0, (0, 255, 0), 2)
                    shapeName = shapeClassification(contour)

            # Erase from Output screen
            if fingers == [0, 1, 1, 1, 0]:
                if annotation:
                    annotation.pop(-1)
                    annotationNumber -= 1
                    shapeName = ""

    for i in range(len(annotation)):
        for j in range(len(annotation[i])):
            if j != 0:
                cv2.line(img, annotation[i][j - 1], annotation[i][j], (0, 0, 255), 12)

    if shapeName:
        cv2.putText(img, shapeName, (380, 360), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

    # Resize the captured frame before displaying it in the output screen
    img = cv2.resize(img, (output_width, output_height))

    # Output screen
    cv2.imshow("Output Screen", img)
    Key = cv2.waitKey(1)

    if Key == ord('q'):
        break
        
        